In [1]:
from tensorflow.keras.datasets import mnist
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
data = mnist.load_data()
x_train = data[0][0]
y_train = data[0][1]
x_train = x_train.reshape(60000, 784)
from sklearn.preprocessing import OneHotEncoder
labels_array = y_train
encoder = OneHotEncoder()
labels_array_reshaped = labels_array.reshape(-1, 1)
encoded_array = encoder.fit_transform(labels_array_reshaped)
encoded_array_dense = encoded_array.toarray()
y_train = encoded_array_dense
x_train.shape , y_train.shape

((60000, 784), (60000, 10))

In [3]:
x_test = data[1][0]
y_test = data[1][1]
x_test = x_test.reshape(10000, 784)
from sklearn.preprocessing import OneHotEncoder
labels_array = y_test
encoder = OneHotEncoder()
labels_array_reshaped = labels_array.reshape(-1, 1)
encoded_array = encoder.fit_transform(labels_array_reshaped)
encoded_array_dense = encoded_array.toarray()
y_test = encoded_array_dense
x_test.shape , y_test.shape

((10000, 784), (10000, 10))

In [4]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) *n_conv2

In [5]:
weights = {
    "wc1" : tf.Variable(tf.random.normal([conv1_k, conv1_k, input_channels, n_conv1])),
    "wc2" : tf.Variable(tf.random.normal([conv2_k, conv2_k, n_conv1, n_conv2])),
    "wh1" : tf.Variable(tf.random.normal([input_size_to_hidden, n_hidden])),
    "wo" : tf.Variable(tf.random.normal([n_hidden, n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random.normal([n_conv1])),
    "bc2" : tf.Variable(tf.random.normal([n_conv2])),
    "bh1" : tf.Variable(tf.random.normal([n_hidden])),
    "bo" : tf.Variable(tf.random.normal([n_out])),
}

In [6]:
def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x, weights, padding="SAME", strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding = "SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1])

In [13]:
def cnn(x, weights, biases,keep_prob):
    x = tf.reshape(x,shape = [-1,input_height,input_width,input_channels])
    conv1 = conv(x,weights['wc1'],biases['bc1'],stride_conv1)
    conv1_pool = maxpooling(conv1,max_pool1_k)
    
    conv2 = conv(conv1_pool,weights['wc2'],biases['bc2'],stride_conv2)
    conv2_pool = maxpooling(conv2,max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool,shape = [-1,input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input,weights['wh1']),biases['bh1'])
    hidden_output_before_droput = tf.nn.relu(hidden_output_before_activation)
    #dropout layer
    hidden_output = tf.nn.dropout(hidden_output_before_droput,keep_prob)
    output = tf.add(tf.matmul(hidden_output,weights['wo']),biases['bo'])
    return output

In [14]:
x = tf.placeholder("float",[None,input_pixels])
y = tf.placeholder(tf.int32,[None,n_out])
keep_prob = tf.placeholder('float')
pred = cnn(x,weights,biases,keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels = y))

In [10]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
for i in range(25):
    c, _ = sess.run([cost,optimize], feed_dict={x:x_train , y:y_train})
    print(c)